In [2]:
import asyncio
import collections.abc
import inspect

# Full Demo: Async Methods

## `__aiter__`, `__anext__`

From [PEP-525](https://www.python.org/dev/peps/pep-0525/#asynchronous-generator-object)

* `obj.__aiter__()` immediately returns an asynchronous generator (note: semantics are slightly different pre-3.5.2)
* `obj.__anext__()` returns an awaitable that will perform one iteration when awaited.
* `obj.asend(val)` returns an awaitable that pushes `val` into `agen` on the next loop

In [3]:
async def async_gen_func(limit):
    """Asynchronous generator function"""
    await asyncio.sleep(0.1)
    n = 0
    while n < limit:
        n += 1
        yield n

        
class AsyncIterable:
    """
    Asynchronous iterable 
    """
    def __init__(self, limit):
        self.n = 0
        self.limit = limit
        
    async def __aiter__(self):
        await asyncio.sleep(0.1)
        while self.n < self.limit:
            self.n += 1
            value = yield self.n
            if value is not None:
                self.n = value
            
            
class AsyncIterator:
    """
    Asynchronous iterator.
    
    object using Python methods
    
    This provides additional flexibility but does not fully 
    replicate the behavior the asynchronous generator above
    which also will support 
      - aiter.asend(value) and 
      - aiter.athrow(exc)      
    """
    def __init__(self, limit):
        self.n = 0
        self.limit = limit
        
    def __aiter__(self):
        # NOTE: not async!
        return self
    
    async def __anext__(self):
        await asyncio.sleep(0.1)
        self.n += 1
        if self.n > self.limit:
            raise StopAsyncIteration
        return self.n

In [4]:
async def demo_ag(ag_callable, *args):
    """
    Exercise some callable that returns an async generator
    """
    print("manual: ", end="")

    iter_ = ag_callable(*args)
    async_gen = type(iter_).__aiter__(iter_)
    running = True
    while running:
        try:
            target = await async_gen.__anext__()
        except StopAsyncIteration:
            running = False
        else:
            # <LOOP BODY>
            print(target, end=", ")
            # </LOOP BODY>
    else:
        # <LOOP ELSE>
        print("done")
        # </LOOP ELSE>           
            
    ##########################################
    print("magic : ", end="")
    
    async for target in async_gen_func(*args):
        # <LOOP BODY>
        print(target, end=", ")
        # </LOOP BODY>
    else:
        # <LOOP ELSE>
        print("done")
        # </LOOP ELSE>

In [5]:
await demo_ag(async_gen_func, 3)

manual: 1, 2, 3, done
magic : 1, 2, 3, done


In [13]:
await demo_ag(AsyncIterable, 4)

manual: 1, 2, 3, 4, done
magic : 1, 2, 3, 4, done


In [14]:
await demo_ag(AsyncIterator, 5)

manual: 1, 2, 3, 4, 5, done
magic : 1, 2, 3, 4, 5, done


## `__aenter__`, `__aexit__`

Akin to `__enter__` and `__exit__` which are invoked in a `with` block, `__aenter__` and `__aexit__` coroutine methods are invoked with `async with`. Their signatures are the same, with the entrance method taking no arguments, and the exit method takes 3 about the exception.

In [17]:
class AsyncContextManager:
    def __init__(self, exclusive_resource=None):
        self.exclusive_resource = exclusive_resource

    async def __aenter__(self):
        self.exclusive_resource
    
    async def __aexit__(self, exc_type, exc_value, tb):
        pass

In [18]:
acm = AsyncContextManager()
acm

try:
    async with acm:
        acm.resource_used = True
        # do something dangerous
        1/0
except Exception as exc:
    print(exc)

division by zero


## `__await__`

Implementing the `__await__` method on a class allows an instance of it to be awaited. However, the `__await__` method itself must be synchronous (not `async def __await__`), and return an iterator.

In [1]:
# future-like object
class FutureLike():
    def __init__(self, sleep=0.1):
        self.t_sleep = sleep

    async def _wrapped(self):
        await asyncio.sleep(self.t_sleep)
        return 1871

    def __await__(self):
        return self._wrapped().__await__()
    
future_like_object = FutureLike()
FLO = future_like_object

assert not inspect.iscoroutine(FLO)
assert not inspect.iscoroutinefunction(FLO)
assert inspect.isawaitable(FLO)
assert isinstance(FLO, collections.abc.Awaitable)

await FLO

NameError: name 'inspect' is not defined

In [370]:
async def native_coroutine_function():
    return 1893

native_coroutine_object = native_coroutine_function()

NCF = native_coroutine_function
NCO = native_coroutine_object

assert not inspect.iscoroutine(NCF)
assert inspect.iscoroutinefunction(NCF)
assert not inspect.isawaitable(NCF)
assert not isinstance(NCF, collections.abc.Awaitable)

assert inspect.iscoroutine(NCO)
assert not inspect.iscoroutinefunction(NCO)
assert inspect.isawaitable(NCO)
assert isinstance(NCO, collections.abc.Awaitable)

await NCO

1893

For historical interest, there is the ` @asyncio.coroutine` decorator which can mark iterators as coroutines. They act identically in typical code (can both be initialized with a call, then awaited), but introspection (via the `inspect` module) can tease them apart. The syntax used here would be compatible with Python 2 *(boos loudly)*

In [362]:
@asyncio.coroutine
def generator_based_coroutine_function():
    yield from asyncio.sleep(0.1)
    return 1904

generator_based_coroutine_object = generator_based_coroutine_function()

GBCF = generator_based_coroutine_function
GBCO = generator_based_coroutine_object

assert not inspect.iscoroutine(GBCF)
assert not inspect.iscoroutinefunction(GBCF)
assert not inspect.isawaitable(GBCF)
assert not isinstance(GBCF, collections.abc.Awaitable)

assert not inspect.iscoroutine(GBCO)
assert not inspect.iscoroutinefunction(GBCO)
assert inspect.isawaitable(GBCO)
assert not isinstance(GBCO, collections.abc.Awaitable) # :/

await GBCO

1904